# 스플래쉬 프로젝트

필요한 library import

In [ ]:
import time
import random
from colorsys import hsv_to_rgb
import board
from digitalio import DigitalInOut, Direction
from PIL import Image, ImageDraw, ImageFont
from adafruit_rgb_display import st7789
import numpy as np

Joystick클래스 선언

In [ ]:
class Joystick:
    def __init__(self):
        self.cs_pin = DigitalInOut(board.CE0)
        self.dc_pin = DigitalInOut(board.D25)
        self.reset_pin = DigitalInOut(board.D24)
        self.BAUDRATE = 24000000

        self.spi = board.SPI()
        self.disp = st7789.ST7789(
                    self.spi,
                    height=240,
                    y_offset=80,
                    rotation=180,
                    cs=self.cs_pin,
                    dc=self.dc_pin,
                    rst=self.reset_pin,
                    baudrate=self.BAUDRATE,
                    )

        # 조이스틱 입력 핀 설정
        self.button_A = DigitalInOut(board.D5)
        self.button_A.direction = Direction.INPUT

        self.button_B = DigitalInOut(board.D6)
        self.button_B.direction = Direction.INPUT

        self.button_L = DigitalInOut(board.D27)
        self.button_L.direction = Direction.INPUT

        self.button_R = DigitalInOut(board.D23)
        self.button_R.direction = Direction.INPUT

        self.button_U = DigitalInOut(board.D17)
        self.button_U.direction = Direction.INPUT

        self.button_D = DigitalInOut(board.D22)
        self.button_D.direction = Direction.INPUT

        self.button_C = DigitalInOut(board.D4)
        self.button_C.direction = Direction.INPUT

        # 백라이트 설정
        self.backlight = DigitalInOut(board.D26)
        self.backlight.switch_to_output()
        self.backlight.value = True

        # 디스플레이 작업 설정
        self.width = self.disp.width
        self.height = self.disp.height

아바타 선언

In [ ]:
class Character:
    def __init__(self, width, height, color="#FF0000", shape='circle', outline="#FF0000"):
        self.shape = shape  # 아바타 모양 (circle, square 등)
        self.state = None # 아바타 상태(정지 또는 이동)
        self.scale = 5 # 초기 크기 설정
        self.speed = 5  # 초기 이동 속도 설정
        self.position = np.array([width/2 - 70 - self.scale, height/2 - self.scale, width/2 - 70 + self.scale, height/2 + self.scale]) # 초기 아바타 위치 설정
        self.outline = outline # 색상(아바타 테두리)
        self.color = color  # 색상(아바타)
        self.path = []  # 경로 저장 리스트
        self.screen_height = height  # 화면 높이 저장
        self.timer_height = 50  # 타이머 직사각형의 높이 (예시: 50px)

    def move(self, command=None):
        if command['move'] == False:
            self.state = None
        else:
            self.state = 'move'
            # 상하 이동
            if command['up_pressed']:  
                self.position[1] -= self.speed
                self.position[3] -= self.speed
            elif command['down_pressed']:
                self.position[1] += self.speed
                self.position[3] += self.speed
            # 좌우 이동
            if command['left_pressed']:
                self.position[0] -= self.speed
                self.position[2] -= self.speed
            elif command['right_pressed']:
                self.position[0] += self.speed
                self.position[2] += self.speed

            # 화면 경계 체크 (찌그러짐 방지)
            width = self.position[2] - self.position[0]
            height = self.position[3] - self.position[1]

            # 상단 직사각형 영역(타이머)을 고려한 경계 설정
            if self.position[0] < 0:  # 왼쪽 경계
                self.position[0] = 0
                self.position[2] = width
            if self.position[2] > 240:  # 오른쪽 경계
                self.position[2] = 240
                self.position[0] = 240 - width

            if self.position[1] < self.timer_height:  # 상단 경계 (타이머 영역을 고려)
                self.position[1] = self.timer_height
                self.position[3] = height + self.timer_height  # 아래쪽도 동일하게 이동
            if self.position[3] > self.screen_height:  # 아래쪽 경계
                self.position[3] = self.screen_height
                self.position[1] = self.screen_height - height

            self.path.append(self.position.copy())  # 현재 위치를 path에 추가

    # 이동 속도 증가 아이템 획득 시
    def increase_speed(self, amount):
        self.speed += amount
    
    # 아바타 크기 증가 아이템 획득 시
    def increase_scale(self, amount):
        self.scale += amount
        self.position[0] -= amount
        self.position[1] -= amount
        self.position[2] += amount
        self.position[3] += amount
        


아이템 선언

In [ ]:
class Item:
    def __init__(self, item_type):
        self.active = False  # 아이템 활성 상태
        self.item_type = item_type  # 아이템 타입 ("speed", "scale")
        self.size = (20, 20)  # 아이템 크기
        self.position = (random.randint(50, 200), random.randint(80, 160))  # 랜덤 위치 생성
        self.spawn_time = random.randint(20, 60)  # 아이템 생성 시간
        self.spawned = False  # 아이템 생성 여부

타이머 출력 함수

In [ ]:
def draw_timer(draw, player_score, ai_score, remaining_time, font, width):
    timer_text = f"{int(remaining_time)}s"
    
    # 텍스트 크기 계산
    text_bbox = draw.textbbox((0, 0), timer_text, font=font)
    text_width, text_height = text_bbox[2] - text_bbox[0], text_bbox[3] - text_bbox[1]

    # 타이머 텍스트 위치 계산 (화면 상단 중앙)
    text_x = (width - text_width) / 2 
    text_y = 8  

    # 원하는 영역 크기 (가로 화면 길이에 맞게, 세로 50px)
    box_width = width
    box_height = 50

    # 직사각형(타이머 박스) 영역의 좌측 상단과 우측 하단 좌표 계산
    timer_area_top_left = (0, text_y - (box_height - text_height) / 2)
    timer_area_bottom_right = (box_width, timer_area_top_left[1] + box_height)
    
    # 노란색 타이머 박스 그리기
    draw.rectangle([timer_area_top_left, timer_area_bottom_right], fill=(255, 255, 0))  # 노란색

    # 타이머 텍스트 그리기
    draw.text(
        (text_x, text_y),  # 텍스트 위치
        timer_text,
        font=font,
        fill=(0, 0, 0)  # 검정색
    )

    # 점수 폰트 크기
    score_font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 20)  # 폰트 크기 조정

    # 플레이어 점수 텍스트 (좌측)
    player_score_text = f"{player_score}"
    player_text_x = 30  # 왼쪽 여백
    player_text_y = 12

    # AI 점수 텍스트 (우측)
    ai_score_text = f"{ai_score}"
    ai_text_x = 195  # 오른쪽 여백
    ai_text_y = 12

    # 점수 텍스트 그리기
    # 플레이어
    draw.text(
        (player_text_x, player_text_y),
        player_score_text,
        font=score_font,
        fill=(0, 0, 0)  # 검정색
    )
    
    # AI
    draw.text(
        (ai_text_x, ai_text_y),
        ai_score_text,
        font=score_font,
        fill=(0, 0, 0)  # 검정색
    )

    # 플레이어 점수 옆에 빨간색 원 그리기(누구 점수인지 구분하기 용도)
    draw.ellipse(
        (10, 15, 25, 30),
        fill=(255, 0, 0),  # 빨간색
        outline=(0, 0, 0)  # 검은색 테두리
    )

    # AI 점수 옆에 파란색 원 그리기
    draw.ellipse(
        (175, 15, 190, 30),
        fill=(0, 0, 255),  # 파란색
        outline=(0, 0, 0)  # 검은색 테두리
    )


joystick, Image, Draw등 게임에 사용할 각종 객체 생성

In [ ]:
joystick = Joystick()
my_image = Image.new("RGB", (joystick.width, joystick.height)) #도화지!
my_draw = ImageDraw.Draw(my_image) #그리는 도구!

# 게임 내에서 점수 계산
my_circle_pixel_count = 0
ai_circle_pixel_count = 0

# 타이머 변수
timer_start = time.time()  # 현재 시간 저장
total_time = 60          # 제한 시간(현재 60초로 설정)

# 붉은색 원 생성 / 잔상이 남지 않는 코드 & 대각선 이동 가능
my_circle = Character(joystick.width, joystick.height)

# 푸른색 원 생성
ai_circle = Character(joystick.width, joystick.height)
ai_circle.color = "#0000FF"  # 푸른색
ai_circle.outline = "#0000FF"
ai_circle.position = np.array([
    joystick.width / 2 + 65,  # 붉은 원보다 오른쪽으로 이동
    joystick.height / 2 - 5,
    joystick.width / 2 + 75,
    joystick.height / 2 + 5
])

# 플레이어와 AI 원의 중심 좌표를 저장할 변수
player_center = (0, 0)
ai_center = (0, 0)

# 원의 중심 좌표를 계산하는 함수
def calculate_center(position):
    x1, y1, x2, y2 = position
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    return (center_x, center_y)

# AI 관련 변수
last_ai_move_time = time.time()  # 마지막으로 AI가 움직인 시간
last_ai_direction = 'left'  # 초기 방향 설정
ai_delay = 0.5  # 방향 변경 딜레이 (0.5초)

my_draw.rectangle((0, 0, joystick.width, joystick.height), fill = (255, 255, 255, 100))

# 폰트 설정
fnt = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 30)

버튼 클릭 설계

In [ ]:
# 버튼 색상 설정
button_A_fill = "#00FF00"  
button_B_fill = "#00FF00"
button_outline = "#000000"
button_radius = 20

# 버튼 위치
button_A_pos = (160, 210)
button_B_pos = (210, 210)

# 버튼 그리는 함수
def draw_button(draw, center, fill, label, font, outline, pressed=False):
    x, y = center
    radius = button_radius
    # 버튼 원 그리기
    draw.ellipse(
        (x - radius, y - radius, x + radius, y + radius),
        outline=outline,
        fill=fill,
    )

    # 텍스트 크기 계산 (최신 방식 사용)
    text_bbox = draw.textbbox((0, 0), label, font=font)
    text_width, text_height = text_bbox[2] - text_bbox[0], text_bbox[3] - text_bbox[1]

    # 텍스트를 버튼 중앙에 그리기
    draw.text(
        (x - text_width / 2 + 1, y - text_height / 2 - 4),
        label,
        font=font,
        fill=outline,
    )
    
# 60초 동안 랜덤한 시간 생성
ai_skill_time_1 = random.randint(10, 59)  # AI가 첫 번째 스킬 사용 시간
ai_skill_time_2 = random.randint(10, 59)  # AI가 두 번째 스킬 사용 시간

# 플레이어 스킬 사용 여부 추적 변수
button_a_used = False  # A 버튼이 사용되었는지 여부
button_b_used = False  # B 버튼이 사용되었는지 여부

# AI 스킬 사용 여부 추적 변수
ai_skill_used_1 = False
ai_skill_used_2 = False

아이템 생성 코드

In [ ]:
# 아이템 생성
item_list = [
    Item("speed") for _ in range(3)
] + [
    Item("scale") for _ in range(3)
]  # 총 6개의 아이템 (3개 speed, 3개 scale)

In [ ]:
while True:
    command = {'move': False, 'up_pressed': False, 'down_pressed': False, 'left_pressed': False, 'right_pressed': False}

    # 플레이어 움직임 처리
    if not joystick.button_U.value:
        command['up_pressed'] = True
        command['move'] = True

    if not joystick.button_D.value:
        command['down_pressed'] = True
        command['move'] = True

    if not joystick.button_L.value:
        command['left_pressed'] = True
        command['move'] = True

    if not joystick.button_R.value:
        command['right_pressed'] = True
        command['move'] = True

    my_circle.move(command)

    # AI 움직임 처리
    current_time = time.time()
    if current_time - last_ai_move_time >= ai_delay:  # AI가 방향을 바꿀 시간인지 체크
        last_ai_direction = random.choice(['up', 'down', 'left', 'right'])  # 방향 변경
        last_ai_move_time = current_time  # 방향 변경 시간 갱신

    # AI의 현재 방향 유지하며 이동
    ai_command = {'move': True, 'up_pressed': False, 'down_pressed': False, 'left_pressed': False, 'right_pressed': False}
    if last_ai_direction == 'up':
        ai_command['up_pressed'] = True
    elif last_ai_direction == 'down':
        ai_command['down_pressed'] = True
    elif last_ai_direction == 'left':
        ai_command['left_pressed'] = True
    elif last_ai_direction == 'right':
        ai_command['right_pressed'] = True

    ai_circle.move(ai_command)  # 랜덤 이동 처리

    # 타이머 계산
    remaining_time = total_time - (time.time() - timer_start)
    if remaining_time < 0:
        remaining_time = 0

    # 붉은색 원과 푸른색 원을 동시에 그리기 (겹침 없이)
    my_draw.ellipse(tuple(my_circle.position), outline=my_circle.outline, fill=my_circle.color)
    my_draw.ellipse(tuple(ai_circle.position), outline=ai_circle.outline, fill=ai_circle.color)
    
    # 게임 루프에서 매번 원의 중심 좌표를 갱신
    player_center = calculate_center(my_circle.position)
    ai_center = calculate_center(ai_circle.position)
    
    # 게임 루프 내에서 AI 스킬 시간 체크
    if (remaining_time <= ai_skill_time_1) and not ai_skill_used_1:
        # AI가 스킬을 사용한 시간과 일치하면 직사각형을 그린다
        ai_skill_used_1 = True  # 스킬 사용했다고 설정

        # 직사각형 그리기
        rect_top = ai_center[1] - 20
        rect_bottom = ai_center[1] + 20
        rect_left = 0  
        rect_right = joystick.width 

        # 직사각형을 푸른색으로 그리기 (AI 스킬 효과)
        my_draw.rectangle(
            [rect_left, rect_top, rect_right, rect_bottom], 
            outline=(0, 0, 255),  # 푸른색
            fill=(0, 0, 255)      # 푸른색
        )
        
    if (remaining_time <= ai_skill_time_2) and not ai_skill_used_2:
        # AI가 스킬을 사용한 시간과 일치하면 직사각형을 그린다
        ai_skill_used_2 = True  # 스킬 사용했다고 설정

        # 직사각형 그리기
        rect_top = 50
        rect_bottom = joystick.height
        rect_left = ai_center[0] - 20 
        rect_right = ai_center[0] + 20 

        # 직사각형을 푸른색으로 그리기 (AI 스킬 효과)
        my_draw.rectangle(
            [rect_left, rect_top, rect_right, rect_bottom], 
            outline=(0, 0, 255),  # 푸른색
            fill=(0, 0, 255)      # 푸른색
        )
    
    # A 버튼 상태 및 그리기
    if not joystick.button_A.value and not button_a_used:  # A 버튼을 처음 눌렀을 때
        button_a_used = True  # 버튼을 사용했다고 설정
        # 직사각형 그리기
        rect_top = player_center[1] - 20 
        rect_bottom = player_center[1] + 20
        rect_left = 0  
        rect_right = joystick.width 

        # 직사각형 그리기
        my_draw.rectangle(
            [rect_left, rect_top, rect_right, rect_bottom], 
            outline=(255, 0, 0),
            fill=(255, 0, 0)
        )

    # A 버튼의 색상 결정
    if button_a_used:
        button_a_fill = "#808080"  # 버튼을 이미 눌렀으므로 회색으로 고정
    else:
        if not joystick.button_A.value:
            button_a_fill = "#808080"  # 버튼을 눌렀을 때 회색
        else:
            button_a_fill = "#FFFFFF"  # 기본 흰색 (버튼이 눌리지 않았을 때)

    # A 버튼 그리기
    A_pressed = button_a_used  # 버튼이 이미 눌린 상태로 설정
    draw_button(my_draw, button_A_pos, button_a_fill, "A", fnt, button_outline, pressed=A_pressed)

    # B 버튼 상태 및 그리기
    if not joystick.button_B.value and not button_b_used:  # B 버튼을 처음 눌렀을 때
        button_b_used = True  # 버튼을 사용했다고 설정
        # 직사각형 그리기 (B 버튼을 눌렀을 때)
        rect_top = 50
        rect_bottom = joystick.height
        rect_left = player_center[0] - 20 
        rect_right = player_center[0] + 20 

        # 직사각형 그리기
        my_draw.rectangle(
            [rect_left, rect_top, rect_right, rect_bottom], 
            outline=(255, 0, 0), 
            fill=(255, 0, 0)
        )

    # B 버튼의 색상 결정
    if button_b_used:
        button_b_fill = "#808080"  # 버튼을 이미 눌렀으므로 회색으로 고정
    else:
        if not joystick.button_B.value:
            button_b_fill = "#808080"  # 버튼을 눌렀을 때 회색
        else:
            button_b_fill = "#FFFFFF"  # 기본 흰색 (버튼이 눌리지 않았을 때)

    # B 버튼 그리기
    B_pressed = button_b_used  # 버튼이 이미 눌린 상태로 설정
    draw_button(my_draw, button_B_pos, button_b_fill, "B", fnt, button_outline, pressed=B_pressed)
    
    # 아이템 생성 
    for item in item_list:
    # 아이템 생성 조건: remaining_time이 spawn_time과 일치할 때, item_count가 0일 경우
        if item.spawned == False and not item.active and remaining_time <= item.spawn_time:
            item.active = True  # 아이템 활성화

        # 아이템이 활성화되었을 때 그리기
        if item.active:
            x, y = item.position
            if(item.item_type == "speed"):
                my_draw.rectangle(
                    [x, y, x + item.size[0], y + item.size[1]],  # 아이템의 좌측 상단과 우측 하단 좌표
                    outline=(0, 0, 0),  # 검은색 테두리
                    fill=(0, 255, 0)  # 초록색 
                )
            else :
                my_draw.rectangle(
                    [x, y, x + item.size[0], y + item.size[1]],  # 아이템의 좌측 상단과 우측 하단 좌표
                    outline=(0, 0, 0),  # 검은색 테두리
                    fill=(0, 255, 255)  # 하늘색 
                )
                
            # 플레이어와 AI의 중심 좌표 확인
            player_x, player_y = player_center  # 플레이어 중심 좌표
            ai_x, ai_y = ai_center  # AI 중심 좌표

            # 플레이어가 아이템에 닿았는지 확인
            if abs(player_x - x) < item.size[0] and abs(player_y - y) < item.size[1]:
                # 플레이어 색상으로 아이템 위치에 사각형을 그리기
                my_draw.rectangle(
                    [x, y, x + item.size[0], y + item.size[1]],  # 아이템의 좌측 상단과 우측 하단 좌표
                    outline=(255, 0, 0),  # 붉은색 테두리
                    fill=(255, 0, 0)  # 붉은색 채우기
                )
                item.active = False  # 아이템 비활성화
                item.spawned = True  # 아이템을 획득했으므로 item_count를 1로 설정
                
                if(item.item_type == "speed"):
                    my_circle.increase_speed(2) # 플레이어 이동 속도 증가
                else :
                    my_circle.increase_scale(5) # 플레이어 아바타 크기 증가

            # AI가 아이템에 닿았는지 확인
            elif abs(ai_x - x) < item.size[0] and abs(ai_y - y) < item.size[1]:
                # AI 색상으로 아이템 위치에 사각형을 그리기
                my_draw.rectangle(
                    [x, y, x + item.size[0], y + item.size[1]],  # 아이템의 좌측 상단과 우측 하단 좌표
                    outline=(0, 0, 255),  # 푸른색 테두리
                    fill=(0, 0, 255)  # 푸른색 채우기
                )
                item.active = False  # 아이템 비활성화
                item.spawned = True  # 아이템을 획득했으므로 item_count를 True로 설정
                
                if(item.item_type == "speed"):
                    ai_circle.increase_speed(2) # 플레이어 이동 속도 증가
                else :
                    ai_circle.increase_scale(5) # 플레이어 아바타 크기 증가

    
    # 붉은색 원 영역의 RGB 픽셀 카운트 (효율적인 방법으로 개선)
    my_image_data = np.array(my_image)
    my_circle_pixel_count = np.sum(np.all(my_image_data == [255, 0, 0], axis=-1))

    # 푸른색 원 영역의 RGB 픽셀 카운트 (효율적인 방법으로 개선)
    ai_circle_pixel_count = np.sum(np.all(my_image_data == [0, 0, 255], axis=-1))

    # 점수 계산
    player_score = my_circle_pixel_count // 100  # 예시: 100 픽셀 단위로 점수 계산
    ai_score = ai_circle_pixel_count // 100  # 예시: 100 픽셀 단위로 점수 계산

    # 타이머 및 점수 그리기
    draw_timer(my_draw, player_score, ai_score, remaining_time, fnt, joystick.width)
    
    # 화면 출력
    joystick.disp.image(my_image)
    
    # 종료 조건 확인 (시간 종료)
    if remaining_time <= 0:
        break

    # 딜레이
    time.sleep(0.01)  # 화면 갱신 주기


게임 종료 후 픽셀 카운트

In [ ]:
def draw_text(draw, text, font, width, height):
    # 텍스트 크기 계산
    text_width, text_height = draw.textsize(text, font=font)

    # 텍스트가 화면에 잘리지 않도록 크기 조정 (여유 공간에 맞게 크기 조정)
    max_text_width = width - 20  # 양옆에 10px씩 여백을 두고
    max_text_height = height - 20  # 상하에 10px씩 여백을 두고

    # 텍스트가 화면에 맞지 않으면 폰트 크기를 줄여서 텍스트를 다시 계산
    while text_width > max_text_width or text_height > max_text_height:
        font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", font.size - 1)
        text_width, text_height = draw.textsize(text, font=font)

    # 텍스트를 화면 중앙에 위치시키기
    text_x = (width - text_width) / 2
    text_y = (height - text_height) / 2

    # 텍스트 그리기
    draw.text((text_x, text_y), text, font=font, fill=(0, 0, 0))  # 검정색 글씨

# 게임 종료 후 픽셀 카운트
my_circle_pixel_count = 0
ai_circle_pixel_count = 0

# 붉은색 원 영역의 RGB 픽셀 카운트 (효율적인 방법으로 개선)
my_image_data = np.array(my_image)
my_circle_pixel_count = np.sum(np.all(my_image_data == [255, 0, 0], axis=-1))

# 푸른색 원 영역의 RGB 픽셀 카운트 (효율적인 방법으로 개선)
ai_circle_pixel_count = np.sum(np.all(my_image_data == [0, 0, 255], axis=-1))

# 점수 100으로 나누기
my_circle_pixel_count //= 100  # 100으로 나눈 값
ai_circle_pixel_count //= 100  # 100으로 나눈 값

# 승패 결정
result_text = ""
if my_circle_pixel_count > ai_circle_pixel_count:
    result_text += f"You Win!\n\nYour score: {my_circle_pixel_count}\nAI's score: {ai_circle_pixel_count}"
else:
    result_text += f"You Lose!\n\nYour score: {my_circle_pixel_count}\nAI's score: {ai_circle_pixel_count}"

# 게임 종료 후 결과 화면에 출력
my_draw.rectangle((0, 0, joystick.width, joystick.height), fill=(255, 255, 255))  # 배경 초기화

# 결과 텍스트 출력 (너비와 높이를 250으로 지정)
draw_text(my_draw, result_text, fnt, joystick.width, joystick.height)

# 결과 화면을 표시
joystick.disp.image(my_image)